**Christian Data Exploration**

This notebook outlines the actions taken to transform the data so that it is ready for model use. Specifically:
- It will convert binary columns like win_loss, OT, Homegame, etc. into 1's and 0's where 1 indicates the symantical presense of "something." 
- Convert numeric string values from NaN to 0.
- Convert conference into a series of binary variables.
- Attempt to identify a solution to the AP top 25 flow-in and out perdicament.
- Shift the dataframe to identify the rank changes.       

**Steps and Workflow**
1. Data setup
2. Data Exploration
    - Explore AP Rank Data
    - Get all Unique Conference Values
3. Data Manipulation
    - Setting up AP Rank Shift
    - Setup all Binary Variables
4. Data Validation Check

**Data Concepts**
- The dataset contains two records for each game: (1) the winner and (2) the looser. I think this is fine--but rankings are zero-sum. 
- AP Ranks only display for the top 25 teams and the AP rank either represents before or after. Eitherway a shift is needed to determine the before and after on each game and create the differential. 

**Outstanding Questons / Data Issues**
- What does "Bye" mean in the opponent field? It seems like perhaps they did not have a game at during this week and perhaps could be dropped? 
- There still seems to be negative values in the week columns -- may want to just drop 2020 from the dataset since it may cause errors. 
- I may need to drop the first week of the season or remove it's shift values since it's pulling in the previous season's information.


**1. Data Setup**

In [ ]:
import pandas as pd
import seaborn as sn

# Set Standard dataframe settings
pd.set_option('display.max_columns', None)
df_clean = pd.read_csv(r'C:\Users\martchfr\OneDrive - Indiana University\Graduate School\MIS\INFO-H 501\Projects\Group-8-Project\03 - Cleaned Data Space\mergedTrainingData.csv')

df_clean.drop(columns =['Unnamed: 0'], inplace=True)
df_clean.sort_values(by=['season','Team','week']).reset_index().head(20)

index  week  season       Team          opponent  code         date  \
0    2238     2    2021  Air Force              Navy  2005  Sat, Sep 11   
1    2600     3    2021  Air Force        Utah State  2005  Sat, Sep 18   
2    3007     4    2021  Air Force  Florida Atlantic  2005  Sat, Sep 25   
3    3454     5    2021  Air Force        New Mexico  2005   Sat, Oct 2   
4    3912     6    2021  Air Force           Wyoming  2005   Sat, Oct 9   
5    4340     7    2021  Air Force       Boise State  2005  Sat, Oct 16   
6    4763     8    2021  Air Force   San Diego State  2005  Sat, Oct 23   
7       0    10    2021  Air Force              Army  2005   Sat, Nov 6   
8     464    11    2021  Air Force    Colorado State  2005  Sat, Nov 13   
9     929    12    2021  Air Force            Nevada  2005  Fri, Nov 19   
10   1406    13    2021  Air Force              UNLV  2005  Fri, Nov 26   
11   1868    14    2021  Air Force        Louisville  2005  Tue, Dec 28   
12   2239     2    2021      Akron            Temple  2006  Sat, Sep 11   
13   3008     4    2021      Akron        Ohio State  2006  Sat, Sep 25   
14   3455     5    2021      Akron              Ohio  2006   Sat, Oct 2   
15   3913     6    2021      Akron     Bowling Green  2006   Sat, Oct 9   
16   4341     7    2021      Akron        Miami (OH)  2006  Sat, Oct 16   
17   4764     8    2021      Akron           Buffalo  2006  Sat, Oct 23   
18   5203     9    2021      Akron        Ball State  2006   Tue, Nov 2   
19      1    10    2021      Akron  Western Michigan  2006   Tue, Nov 9   

   win_loss     OT OT_num   pass   rush    rec  points_allowed  points_scored  \
0         W  False    NaN   49.0   97.0   27.0               3             23   
1         L  False    NaN  182.0  102.0    NaN              49             45   
2         W  False    NaN   70.0  164.0   46.0               7             31   
3         W  False    NaN   33.0  142.0   33.0              10             38   
4         W  False    NaN  110.0  140.0   77.0              14             24   
5         W  False    NaN   59.0  138.0   59.0              17             24   
6         L  False    NaN   58.0   50.0   31.0              20             14   
7         L   True         226.0   68.0  106.0              21             14   
8         W  False    NaN  121.0  151.0   92.0              21             35   
9         W   True      3   23.0  208.0   23.0              39             41   
10        W  False    NaN    NaN  123.0    NaN              14             48   
11        W  False    NaN  252.0   77.0  172.0              28             31   
12        L  False    NaN  109.0   66.0  101.0              45             24   
13        L  False    NaN  115.0   47.0   67.0              59              7   
14        L  False    NaN  175.0   64.0   69.0              34             17   
15        W  False    NaN  177.0  120.0  106.0              20             35   
16        L  False    NaN  291.0   29.0  109.0              34             21   
17        L  False    NaN   66.0  155.0   36.0              45             10   
18        L  False    NaN  331.0   88.0  154.0              31             25   
19        L  False    NaN  305.0   58.0  122.0              45             40   

    point_differential  home_game           CONF  SOR  FPI  SOS   GC  AVGWP  \
0                   20      False  Mountain West   33   66  110   24      6   
1                   -4       True  Mountain West   33   66  110   24      6   
2                   24       True  Mountain West   33   66  110   24      6   
3                   28      False  Mountain West   33   66  110   24      6   
4                   10       True  Mountain West   33   66  110   24      6   
5                    7      False  Mountain West   33   66  110   24      6   
6                   -6       True  Mountain West   33   66  110   24      6   
7                   -7       True  Mountain West   33   66  110   24      6   
8              

**2. Data Exploration - Explore AP Rank Data** 

In [33]:
# Select all rows where AP_rank is not null and then display the first 14 rows sorted by team and week
mask = df_clean['AP_rank'].notna()
df_ap_rank = df_clean[mask]
df_ap_rank.iloc[0:14].sort_values(by=['Team','week']).reset_index()

index  week  season              Team         opponent  code         date  \
0       2    10    2021           Alabama              LSU   333   Sat, Nov 6   
1       7    10    2021            Auburn        Texas A&M     2   Sat, Nov 6   
2       9    10    2021            Baylor              TCU   239   Sat, Nov 6   
3      16    10    2021        Cincinnati            Tulsa  2132   Sat, Nov 6   
4      17    10    2021  Coastal Carolina    Georgia State   324  Sat, Nov 13   
5      27    10    2021      Fresno State  San Diego State   278  Sat, Oct 30   
6      28    10    2021           Georgia         Missouri    61   Sat, Nov 6   
7      33    10    2021           Houston    South Florida   248   Sat, Nov 6   
8      36    10    2021              Iowa     Northwestern  2294   Sat, Nov 6   
9      41    10    2021          Kentucky        Tennessee    96   Sat, Nov 6   
10     43    10    2021         Louisiana             Troy   309  Sat, Nov 13   
11     52    10    2021          Michigan          Indiana   130   Sat, Nov 6   
12     53    10    2021    Michigan State           Purdue   127   Sat, Nov 6   
13     66    10    2021        Notre Dame             Navy    87   Sat, Nov 6   

   win_loss     OT OT_num   pass   rush    rec  points_allowed  points_scored  \
0         W  False    NaN  302.0   18.0  160.0              14             20   
1         L  False    NaN  153.0   69.0   50.0              20              3   
2         L  False    NaN  214.0  125.0  121.0              30             28   
3         W  False    NaN  274.0   43.0  113.0              20             28   
4         L  False    NaN  233.0  128.0  101.0              42             40   
5         W  False    NaN  306.0  186.0  107.0              20             30   
6         W  False    NaN  255.0   41.0   76.0               6             43   
7         W  False    NaN  385.0  130.0  164.0              42             54   
8         W  False    NaN  172.0  141.0   68.0              12             17   
9         L  False    NaN  372.0  109.0  166.0              45             42   
10        W  False    NaN  166.0  102.0   71.0              21             35   
11        W  False    NaN  168.0  168.0  108.0               7             29   
12        L  False    NaN  276.0  136.0   73.0              40             29   
13        W  False    NaN  269.0   95.0  139.0               6             34   

    point_differential  home_game           CONF  SOR  FPI  SOS  GC  AVGWP  \
0                    6       True            SEC    2    2    1   2      3   
1                  -17      False            SEC   48   20    4  29     58   
2                   -2      False         Big 12    7   15   25   6      7   
3                    8       True       American    6   10   54   5      2   
4                   -2       True       Sun Belt   34   45  130  47      5   
5                   10      False  Mountain West   30   53   95  37     19   
6                   37       True            SEC    1    1    3   1      1   
7                   12      False       American   17   37   78  27     10   
8                    5      False        Big Ten   16   32   23  33     54   
9                   -3       True            SEC   19   28   32  15     15   
10                  14      False       Sun Belt   11   48  101  34     12   
11                  22       True        Big Ten    3    4    5   3      4   
12                 -11      False        Big Ten    5   19   13  19     39   
13                  28       True     FBS Indep.   10    6   43   9     11   

    AP_rank  opponent_rank  rank_change  
0       3.0            NaN         -1.0  
1      12.0           13.0         -9.0  
2      14.0            NaN         -4.0  
3       2.0            NaN          0.0  
4      21.0            NaN         -3.0  
5      25.0            NaN         -1.0  
6       1.0            NaN          0.0  
7      20.0            NaN         -6.0  
8      19.0            

**2. Data Exploration - Get all Unique Conference Values**

These values will be used to create a set of binary coolumns for each conference.

In [13]:
# Groups by conference and counts unique teams in each conference.
df_clean.groupby('CONF')['Team'].nunique().reset_index(name='unique_teams')

# Create the conference list for binary column creation
conference_list = []

for conf in df_clean['CONF']:
    if conf not in conference_list:
        conference_list.append(conf)

print(conference_list)

['Mountain West', 'MAC', 'SEC', 'Pac-12', 'FBS Indep.', 'Big 12', 'ACC', 'CUSA', 'American', 'Sun Belt', 'Big Ten']


**3. Data Manipulation - Setup AP Rank Shift and AP Rank Differential**

In [23]:
# Define main dataframe, sorted by team and week for manipulation
df_base = df_clean.sort_values(by=['Team','season','week']).reset_index()

df_base["Previous_AP_Rank"] = df_base.groupby('Team')['AP_rank'].shift(1)
df_base['AP_Rank_Differential'] = df_base['AP_rank'] - df_base['Previous_AP_Rank']

mask = df_base['Team'] == 'Alabama'

df_base[mask].head(14)

# Check on the Alabama Crimson Tide for the first 14 weeks to determien if the previous AP rank shift is working correctly


index  Unnamed: 0  week  season     Team           opponent  code  \
85   2601        2602     3    2021  Alabama            Florida   333   
86   3009        3010     4    2021  Alabama      Southern Miss   333   
87   3456        3457     5    2021  Alabama           Ole Miss   333   
88   3914        3915     6    2021  Alabama          Texas A&M   333   
89   4342        4343     7    2021  Alabama  Mississippi State   333   
90   4765        4766     8    2021  Alabama          Tennessee   333   
91      2           3    10    2021  Alabama                LSU   333   
92    466         467    11    2021  Alabama   New Mexico State   333   
93    931         932    12    2021  Alabama           Arkansas   333   
94   1407        1408    13    2021  Alabama             Auburn   333   
95   1869        1870    14    2021  Alabama            Georgia   333   
96   2150        2151    15    2021  Alabama         Cincinnati   333   
97   2218        2219    16    2021  Alabama            Georgia   333   
98   2325        2326     2    2022  Alabama              Texas   333   

           date win_loss     OT OT_num   pass   rush    rec  points_allowed  \
85  Sat, Sep 18        W  False    NaN  240.0   78.0   61.0              29   
86  Sat, Sep 25        W  False    NaN  313.0  110.0  105.0              14   
87   Sat, Oct 2        W  False    NaN  241.0  171.0   65.0              21   
88   Sat, Oct 9        L  False    NaN  369.0  147.0  146.0              41   
89  Sat, Oct 16        W  False    NaN  348.0   78.0  117.0               9   
90  Sat, Oct 23        W  False    NaN  371.0  107.0  123.0              24   
91   Sat, Nov 6        W  False    NaN  302.0   18.0  160.0              14   
92  Sat, Nov 13        W  False    NaN  270.0   99.0  158.0               3   
93  Sat, Nov 20        W  False    NaN  559.0  122.0  190.0              35   
94  Sat, Nov 27        W   True      4  317.0   71.0  150.0              22   
95   Sat, Dec 4        W  False    NaN  421.0   55.0  184.0              24   
96  Fri, Dec 31        W  False    NaN  181.0  204.0   66.0               6   
97  Mon, Jan 10        L  False    NaN  369.0   68.0  102.0              33   
98  Sat, Sep 10        W  False    NaN  213.0   97.0   74.0              19   

    points_scored  point_differential  home_game CONF  SOR  FPI  SOS  GC  \
85             31                   2      False  SEC    2    2    1   2   
86             63                  49       True  SEC    2    2    1   2   
87             42                  21       True  SEC    2    2    1   2   
88             38                  -3      False  SEC    2    2    1   2   
89             49                  40      False  SEC    2    2    1   2   
90             52                  28       True  SEC    2    2    1   2   
91             20                   6       True  SEC    2    2    1   2   
92             59                  56       True  SEC    2    2    1   2   
93             42                   7       True  SEC    2    2    1   2   
94             24                   2      False  SEC    2    2    1   2   
95             41                  17       True  SEC    2    2    1   2   
96             27                  21       True  SEC    2    2    1   2   
97             18                 -15       True  SEC    2    2    1   2   
98             20                   1      False  SEC    4    2    9   3   

    AVGWP  AP_rank  opponent_rank  rank_change  Previous_AP_Rank  \
85      3      1.0            9.0          0.0               NaN   
86      3      1.0            NaN          0.0               1.0   
87      3      1.0           12.0          0.0               1.0   
88      3      1.0            NaN          0.0               1.0   
89      3      5.0            NaN          4.0               1.0   
90      3      4.0            NaN         -1.0               5.0   
91      3      3.0            NaN         -1.0               4.0   
92      3      3.0            NaN    

**3. Data Manipulation - Setup all Binary Variables**

In [106]:
# Note -- may need to remove "Bye" / non-game weeks prior to engaging in this manipulation

# Create binary columns for each conference in the conference list
for conf in conference_list:
    df_base[conf] = (df_base['CONF'] == conf).astype(int)

# Creating the binary columns
df_base['win_loss'] = (df_base['win_loss'] == 'W').astype(int)
df_base['OT'] = (df_base['OT'] == True).astype(int)
df_base['home_game'] = (df_base['home_game'] == True).astype(int)

df_base.head()

index               team         date        opponent  win_loss  OT OT_num  \
0   7267  Air Force Falcons   Sat, Oct 3            Navy         1   0    NaN   
1   7268  Air Force Falcons  Sat, Oct 10             Bye         0   0    NaN   
2   7266  Air Force Falcons  Sat, Oct 17             Bye         0   0    NaN   
3   7270  Air Force Falcons  Sat, Oct 24  San José State         0   0    NaN   
4   7271  Air Force Falcons  Sat, Oct 31     Boise State         0   0    NaN   

   pass   rush   rec  points_allowed  points_scored  point_differential  \
0  41.0  118.0  29.0               7             40                  33   
1   0.0    0.0   0.0               0              0                   0   
2   0.0    0.0   0.0               0              0                   0   
3  92.0   60.0  56.0              17              6                 -11   
4  38.0  112.0  38.0              49             30                 -19   

   home_game  week  season           CONF  SOR  FPI AP/CFP  SOS  GC  AVGWP  \
0          0    -2    2020  Mountain West   83   74     --  115  66     40   
1          0    -1    2020  Mountain West   83   74     --  115  66     40   
2          0     0    2020  Mountain West   83   74     --  115  66     40   
3          0     1    2020  Mountain West   83   74     --  115  66     40   
4          0     2    2020  Mountain West   83   74     --  115  66     40   

   AP_rank state  code  Previous_AP_Rank  AP_Rank_Differential  Mountain West  \
0      NaN   NaN   NaN               NaN                   NaN              1   
1      NaN   NaN   NaN               NaN                   NaN              1   
2      NaN   NaN   NaN               NaN                   NaN              1   
3      NaN   NaN   NaN               NaN                   NaN              1   
4      NaN   NaN   NaN               NaN                   NaN              1   

   MAC  SEC  Sun Belt  Big 12  American  ACC  CUSA  Big Ten  FBS Indep.  \
0    0    0         0       0         0    0     0        0           0   
1    0    0         0       0         0    0     0        0           0   
2    0    0         0       0         0    0     0        0           0   
3    0    0         0       0         0    0     0        0           0   
4    0    0         0       0         0    0     0        0           0   

   Pac-12  
0       0  
1       0  
2       0  
3       0  
4       0